# Lab 2: ARMA Processes and Stationarity

For this lab session, we'll focus on simulating and estimating ARMA($p$, $q$) models.

## Setup

First, we import all the necessary Python packages/libraries, and set up some configuration:

In [ ]:
import sys, os
from lib import *

dpi = 120
plt.rcParams['figure.dpi'] = dpi
# _=plt.gcf().set_dpi(dpi)

In [ ]:
path_data = 'data/'

from statsmodels.tsa.api import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima_process import ArmaProcess

import numpy.polynomial.polynomial as poly


Then we define some function for estimation, calculation of ACF/PACF, and simulation of ARMA models.

In [ ]:
def calc_ols(y, x, addcon=True):
    """Calculate OLS coefficients from scratch"""
    Nobs = y.shape[0]
    if addcon:
        X = np.c_[np.ones((Nobs,1)), x] # append the [Nobs x 1] columns of ones.
    else:
        X = x
    XX = np.dot(X.T, X) # X'X
    Xy = np.dot(X.T, y) # X'y
    beta_hat = np.linalg.solve(XX, Xy) # solve normal equations to calculate OLS estimates
    resids = y - np.dot(X, beta_hat) # residual eps_hat = y - beta_hat*X 
    return beta_hat, resids, X


### See: https://stackoverflow.com/questions/20701484/why-do-i-get-only-one-parameter-from-a-statsmodels-ols-fit
def _sm_calc_ols(y, x, addcon=True):
    """Wrapper for statsmodels OLS regression"""
    X = sm.add_constant(x) if addcon else x # add a constant if addcon==True
    ols_results = sm.OLS(y,X).fit(cov_type='HC1')  # robust standard errors
    beta_hat = ols_results.params # beta_hat
    resids = ols_results.resid  # resids
    return beta_hat, resids, X


def lag_mat(y,nlags, fill_vals=np.nan):
    """Create a matrix of lags of a given vector"""
    ### Source: https://docs.scipy.org/doc/numpy/reference/generated/numpy.empty.html
    y_lags = np.empty((y.shape[0], nlags+1)) 
    y_lags.fill(fill_vals)
    
    ### Include 0 lag
    for lag in range(nlags + 1):
        ### Source: https://docs.scipy.org/doc/numpy/reference/generated/numpy.roll.html
        ### np.roll --> Elements that roll beyond the last position are re-introduced at the first.
        y_lags[lag:, lag] = np.roll(y, shift=lag)[lag:] 
    return y_lags

### Sample ACF and PACF for a given lag
def calc_acf_lag_ols(y_lags, lag):
    """ACF for a given lag (OLS)"""
    if lag==0: 
        return 1.
    lhs = y_lags[lag:, 0]
    rhs = y_lags[lag:, lag:lag+1]
    beta_hat, _,_ = calc_ols(y=lhs, x=rhs, addcon=True)
    return beta_hat[-1]

def calc_pacf_lag_ols(y_lags, lag):
    """PACF for a given lag (OLS)"""
    if lag==0: 
        return 1.
    lhs = y_lags[lag:, 0]
    ### need y_lags[lag:, 1:lag+1] instead of y_lags[lag:,lag:lag+1] (unlike "calc_acf_lag_ols")
    rhs = y_lags[lag:, 1:lag+1] 
    beta_hat, _,_ = calc_ols(y=lhs, x=rhs, addcon=True)
    return beta_hat[-1]

### ACF and PACF for all lags
def calc_acf_ols(y, nlags):
    """ACF for multiple lags"""
    y_lags = lag_mat(y,nlags)
    acf_list = [calc_acf_lag_ols(y_lags,lag) for lag in range(nlags + 1)]
    return np.array(acf_list)

def calc_pacf_ols(y, nlags):
    """PACF for multiple lags"""
    y_lags = lag_mat(y,nlags)
    pacf_list = [calc_pacf_lag_ols(y_lags,lag) for lag in range(nlags + 1)]
    return np.array(pacf_list)

### Plotting functions
def my_plot_acf(y, nlags=10, ax=None, title_string='', 
                title_fontsize=None, xlabel_string='Time'):
    """Plotting ACF with approx SEs."""
    T = y.shape[0] 
    ### approx SEs: scaling used in asymptotic
    se_approx = 1/np.sqrt(T)
    ### set up figure
    if ax is None:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,5))
    ### ACF
    ax.plot(calc_acf_ols(y, nlags), c='xkcd:true blue', 
            marker='o', markerfacecolor='xkcd:azure')
    ax.fill_between(x=range(0,nlags+1), y1=-1.96*se_approx, y2=1.96*se_approx, facecolor='blue', alpha=0.1)
    ax.set_xlabel(xlabel_string)
    if title_fontsize!=None:
        ax.set_title('${\it ACF}$: ' + title_string, fontsize=title_fontsize)
    else:
        ax.set_title('${\it ACF}$: ' + title_string)
    
def my_plot_pacf(y, nlags=10, ax=None, title_string='', 
                 title_fontsize=None, xlabel_string='Time'):
    """Plotting PACF with approx SEs."""
    T = y.shape[0]
    # approx SEs: scaling used in aysmptotic
    se_approx = 1/np.sqrt(T)
    # set up figure
    if ax is None:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,5))
    # PACF
    ax.plot(calc_pacf_ols(y, nlags), c='xkcd:true blue', 
            marker='o', markerfacecolor='xkcd:azure')
    ax.fill_between(x=range(0,nlags+1), y1=-1.96*se_approx, y2=1.96*se_approx, facecolor='blue', alpha=0.1)
    ax.set_xlabel(xlabel_string)
    if title_fontsize!=None:
        ax.set_title('${\it PACF}$: ' + title_string, fontsize=title_fontsize)
    else:
        ax.set_title('${\it PACF}$: ' + title_string)

def simulate_arma(ar=[], ma=[], nsample=100, burnin=0, paths=1, rnd=np.random.randn):
    """Simulate ARMA data
       Assumption: White noise shocks are Gaussian.
    """
    ### numpy arrays are reversed for easier indexing:
    ar, ma = np.array(ar[::-1]), np.array(ma[::-1])
    ### Orders (does not include zero lag)
    p, q = len(ar), len(ma)
    max_order = max(p,q)
    
    ### Total number of sample size
    Nsim = nsample + burnin
    ### "Standard" Guassian shocks: Normal(0,1)
    eps = np.random.randn(paths, Nsim)
    eps = rnd(paths, Nsim)

    ### Initialize t < 0 with zeros
    eps = np.concatenate((np.zeros((paths, max_order)), eps), axis=1)
    y = np.zeros((paths, Nsim + max_order))
    
    ### Loop to construct the ARMA processes recursively.
    for tt in range(max_order, Nsim + max_order):
        y[:, tt] = np.sum(y[:, tt-p:tt]*ar, axis=1) + np.sum(eps[:,tt-q:tt]*ma, axis=1) + eps[:,tt]
    
    ### Drop initial zeros and burnin and transpose for plotting.
    y = y[:, max_order + burnin:].T
    return y

## Simulating ARMA models using `simulate_arma()`

Below, we generate and AR(1) and a White-Noise with our "own" code of ARMA simulations from the above functions. Note we start at $y_0=0$.

In [ ]:
### 1. White Noise 
y_wn = simulate_arma(nsample=100)

### 2. AR(1) with persistence = 0.75
phi1_model1 = 0.75
y_ar1_model1 = simulate_arma(ar=[phi1_model1], nsample=250)

fig1, axes1 = plt.subplots(nrows=2, ncols=1, figsize=(12,10))
_=axes1[0].plot(y_wn)
_=axes1[0].set_xlabel('Time', fontsize=16)
_=axes1[0].set_title('White Noise', fontsize=18)
_=axes1[0].axhline(y=0, linewidth=0.4)

_=axes1[1].plot(y_ar1_model1)
_=axes1[1].set_xlabel('Time', fontsize=16)
_=axes1[1].set_title('AR($1$)' + ' with $\phi$ = ' + str(phi1_model1) , fontsize=18)
_=axes1[1].axhline(y=0, linewidth=0.4)

fig1.tight_layout()

## Estimating regression coefficients of AR(1) processes 

Let's simulate an AR(1) process (1000 iterations with 100 timesteps) and estimate the autoregressive coefficient using OLS. 
Is there evidence of bias in the coefficient estimates? 
Let's plot the empirical distribution of $\sqrt{T}(\hat{\phi}-\phi)$ based on our simulated estimates.

In [ ]:
import seaborn as sns # for density plot
N=1000
T=100
betas = []
phi1_model1 = 0.99
for i in range(N):
    y_ar1_model1 = simulate_arma(ar=[phi1_model1], nsample=T)
    betas.append(calc_ols(y_ar1_model1[1:],y_ar1_model1[0:-1])[0][1][0])
transformed_betas = np.sqrt(T)*(np.array(betas)-phi1_model1)
        
fig, axes = plt.subplots(2, 1, figsize=(10, 10))
axes[0].hist(transformed_betas, bins=30, color='skyblue', edgecolor='black')
axes[0].set_title('Histogram')
sns.kdeplot(data=transformed_betas, ax=axes[1], color='red', linewidth=2)
axes[1].set_title('Density Plot')
print("Mean:", np.mean(betas))

## Theoretical properties of ARMA($p$,$q$) models

**Recall:** A stochastic process process $\{x_t\}$ is an ARMA($p$,$q$) process (autoregressive, moving average process of order $p$ and $q$, respectively) if we have

$$
x_t = \mu + \phi_1 x_{t-1} + \phi_2 x_{t-2} + ... \phi_p x_{t-p} + \epsilon_{t} + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} +  ...  + \theta_q \epsilon_{t-q}
$$

where $\epsilon_{t}$ is white noise, i.e. $E[\epsilon_{t}]=0$ and $Var[\epsilon_{t}]=\sigma^2$, and without loss of generality, we set $\mu$=0.

If the coefficients $\phi_i \equiv 0$, then the ARMA($p$,$q$) process collapses to an MA($q$) process. 

Similarly, if $\theta_i\equiv 0$ then the ARMA($p$,$q$) process collapses to an AR($p$) process.

A slightly more general formulation also used is given by the expression


$$
(x_t - \phi_1 x_{t-1} - \phi_2 x_{t-2} + ... \phi_p x_{t-p})= \epsilon_{t} + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} +  ...  + \theta_q \epsilon_{t-q}
$$

$$
\Phi(L) x_t = \Theta(L) \epsilon_{t}
$$

$$
(1-\lambda_{\phi,1} L) ...  (1-\lambda_{\phi,p} L) x_t = (1-\lambda_{\theta,1} L) ...  (1-\lambda_{\theta,p} L) \epsilon_{t}
$$

We will now analyze the properties of ARMA models with the class `ArmaProcess` from the `statsmodel` package.

Notation:
- phi: AR coefficients
- lambda_ar: factors in (1-lambda_ar_1 L)...(1-lambda_ar_p L)
- r_phi=1/lambda_ar: roots of AR polynomial

In [ ]:
from statsmodels.tsa.arima_process import ArmaProcess

In [ ]:
def arma_from_coeff(phi, theta):
    if type(phi)==list:
        phi = np.array(phi)
    if type(theta)==list:
        theta = np.array(theta)

    phi_poly = np.r_[1, -phi]
    theta_poly = np.r_[1, theta]

    # arma_process = ArmaProcess([1], [1, -.7, .1])
    arma_process = ArmaProcess(phi_poly, theta_poly)

    _phi, _r_phi, _lam_phi = arma_process.arcoefs, arma_process.arroots, 1/arma_process.arroots
    _theta, _r_theta, _lam_theta = arma_process.macoefs, arma_process.maroots, 1/arma_process.maroots

    if len(_r_phi)==0:
        _r_phi = np.array([np.inf])
        _lam_phi = np.array([0])
    if len(_r_theta)==0:
        _r_theta = np.array([np.inf])
        _lam_theta = np.array([0])
        
    print('\nAR:\t phi=', _phi, '\tlambda_phi=',_lam_phi)
    print('AR:\t theta=', _theta, '\tlambda_theta=',_lam_theta,'\n')

    return arma_process, _phi, _theta, _lam_phi, _lam_theta


def arma_from_lambda(lam_phi, lam_theta):
    if type(lam_phi)==list:
        lam_phi = np.array(lam_phi)
    if type(lam_theta)==list:
        lam_theta = np.array(lam_theta)
        
    r_phi = 1/lam_phi
    r_theta = 1/lam_theta    
        
    if len(r_phi)==0:
        r_phi = np.array([np.inf])
        lam_phi = np.array([0])
    if len(r_theta)==0:
        r_theta = np.array([np.inf])
        lam_theta = np.array([0])

    arma_process = ArmaProcess.from_roots(arroots=r_phi, maroots=r_theta)

    _phi, _r_phi, _lam_phi = arma_process.arcoefs, arma_process.arroots, 1/arma_process.arroots
    _theta, _r_theta, _lam_theta = arma_process.macoefs, arma_process.maroots, 1/arma_process.maroots

    if len(_r_phi)==0:
        _r_phi = np.array([np.inf])
        _lam_phi = np.array([0])
    if len(_r_theta)==0:
        _r_theta = np.array([np.inf])
        _lam_theta = np.array([0])

    print('\nAR:\t phi=', _phi, '\tlambda_phi=',_lam_phi)
    print('AR:\t theta=', _theta, '\tlambda_theta=',_lam_theta,'\n')

    return arma_process, _phi, _theta, _lam_phi, _lam_theta

In [ ]:
### Start with lambdas (1/roots)

lam_phi = np.array([0.75,0.2])
lam_theta = np.array([0.5, -0.2])

arma_process, phi, theta, lam_phi, lam_theta = arma_from_lambda(lam_phi, lam_theta)

_ = arma_from_coeff(phi, theta)

In [ ]:
### Start with phi and theta

phi = np.array([0.95, -0.15])
theta = np.array([-0.3, -0.1])

arma_process, phi, theta, lam_phi, lam_theta = arma_from_coeff(phi, theta)

_ = arma_from_lambda(lam_phi, lam_theta)

In [ ]:
### AR(2)

_lam_phi = np.array([0.75,0.2])
_lam_theta = np.array([])

arma_process, phi, theta, lam_phi, lam_theta = arma_from_lambda(_lam_phi, _lam_theta)

print('Class: ',arma_process,'\n')
methods = dir(arma_process)
methods = [x for x in methods if not '__' in x]
print('Methods:', methods,'\n')

### useful methods of the ArmaProcess class
arma_process.arcoefs
arma_process.macoefs

arma_process.arroots
arma_process.maroots

arma_process.isstationary

print(arma_process.arpoly)
print(arma_process.mapoly)

arma_process.arma2ma(20)
# _=plt.plot(arma_process.arma2ma(20))

In [ ]:
_=plt.plot(arma_process.acf(20))

In [ ]:
_=plt.plot(arma_process.impulse_response(20))

In [ ]:
### Function: arma_analysis(arma_process, plot=True, H=20)
### returns ACF, IRF 

def arma_analysis(arma_process, plot=True, H=20, plot_pacf=False):   
    
    print('-----------------------------')
    print('ACF and IRF of ARMA processes')
    print('-----------------------------\n')

    print('AR phi = \t',arma_process.arcoefs)
    print('MA theta = \t',arma_process.macoefs,'\n')

    print('AR roots = \t',arma_process.arroots)
    print('AR lambdas = \t',1/arma_process.arroots)
    print('MA phi = \t',arma_process.maroots)
    print('MA lambdas = \t',1/arma_process.maroots,'\n')
    
    ### use methods of `ArmaProcess` object
    ### ACF
    acf = arma_process.acf(lags=H)
    print('ACF:\t',['{:.2f}'.format(i) for i in acf[:10]],'\n')
    if plot:
        _=plt.figure()
        _=plt.title('ACF')
        _=plt.plot(acf, ls='-', marker='o', ms=4)

        ### Comparison: ACF of AR(1) with phi=acf(1)
        phi1 = arma_process.acf(lags=2)[1]
        acf_ar1 = phi1**np.arange(0,H)
        _=plt.plot(acf_ar1, ls='--', marker='o', ms=4)
        plt.show()
    
    ### Impulse response function
    ### Equivalent: MA representation

    # print('IRF:\n',['{:.2f}'.format(i) for i in arma_process.impulse_response(10)],'\n')
    # print(['{:.2f}'.format(i) for i in arma_process.arma2ma(lags=10)],'\n')
    irf = arma_process.impulse_response(H)
    print('IRFF:\t',['{:.2f}'.format(i) for i in irf[:10]],'\n')

    if plot:
        _=plt.figure()
        _=plt.title('IRF')
        _=plt.plot(irf, marker='o', ms=4)

        ### Comparison: ACF of AR(1) with phi=acf(1)
        acf_ar1 = phi1**np.arange(0,H)
        _=plt.plot(acf_ar1, ls='--', marker='o', ms=4)
        plt.show()


        ### Compare ACF and IRF
        _=plt.figure()
        _=plt.title('ACF vs. IRF')
        _=plt.plot(arma_process.impulse_response(H), marker='o', ms=4)
        _=plt.plot(arma_process.acf(lags=H), ls='--', marker='o', ms=4)
        plt.show()
        
    ### PACF
    if plot_pacf:
        pacf = arma_process.acf(lags=H)
        print('PACF:\t',['{:.2f}'.format(i) for i in pacf[:10]],'\n')
        if plot:
            _=plt.figure()
            _=plt.title('PACF')
            _=plt.plot(acf, ls='-', marker='o', ms=4)
            plt.show()
    
    
    return acf, irf

## Exercise 1

Use the function `arma_analysis()` to analyze the following models:

1. ARMA(1,2)
$$
x_t =  0.5 x_{t-1} +  \epsilon_{t} + 1.2 \epsilon_{t-1} + -0.4 \epsilon_{t-2} 
$$
2. AR(2)
$$
x_t =  0.5 x_{t-1} + 0.4 x_{t-2} +  \epsilon_{t}  
$$
3. AR(2)
$$
x_t =  0.5 x_{t-1} - 0.4 x_{t-2} +  \epsilon_{t}  
$$ 

## Exercise 2

Compare the ACFs and IRFs of the following two processes:

(a) an AR(1) process with $\phi_1 = 0.5$

(b) an ARMA(1,2) process with $\phi_1 = 1.25$, $\phi_2 = -0.375$, $\theta_1 = -0.74$.

Why are they so similar?


### Discussion: common zeroes

What is particularly interesting about the ARMA($2$,$1$) process shown above? Let's write down the process using factored lag polynomials for both its AR and MA components:

$$
x_t = 1.25 x_{t-1} - 0.375 x_{t-2} + \epsilon_{t} - 0.74 \epsilon_{t-1}
$$
$$
(1 - 1.25 L - 0.375 L^2)x_{t} = (1 -  0.74 L)\epsilon_{t}
$$
$$
(1 - 0.75 L)(1 - 0.5 L)x_{t} = (1 -  0.74 L)\epsilon_{t}
$$

which gives us the AR roots $\frac{1}{\lambda_1} = 1.3333 $ and $\frac{1}{\lambda_2} = 2$. The (factored) MA lag polynomial $\Theta(L)$ has the single root $\frac{1}{\zeta_1}\approx 1.35$

Thus the ARMA($2$,$1$) almost exhibits what we call a "common factor". That is,  $\Phi(L)$ and $\Theta(L)$ almost have a **common zero**. The AR factor $(1 - 0.75 L)$ is almost identical to the MA factor $(1 -  0.74 L)$, that is, they "almost" cancel each other out. 

Ifwe had $\theta_1 = -0.75$ and the MA factor was $(1-0.75L)$, the process would have a common root of $4/3$ (outside the unit circle). Since the factor $(1-0.75L)$ cancels out, the process would be an **AR(1) process**. 

**Note:** In practice, for small $T$, it may be difficult to "identify" the data generating process. So we would never have an exactly common zero, and usually an approximately common zero. We may still want to remove it.

In [ ]:
## We can also see the similar properties by simulating from these two processes and comparing the mean of the OLS estimates for an AR(1) model

phi1 = np.array([ 0.5])
theta1 = np.array([])
phi2 = np.array([1.25, -0.375])
theta2 = np.array([-0.74])
arma_process1, phi1, theta1, lam_phi1, lam_theta1 = arma_from_coeff(phi1, theta1)
arma_process2, phi2, theta2, lam_phi2, lam_theta2 = arma_from_coeff(phi2, theta2)
T, N = 100, 10000
sim1 = arma_process1.generate_sample(nsample=(T,N))
sim2 = arma_process2.generate_sample(nsample=(T,N))
betas1 = []
betas2 = []    
for i in range(N):
    y_ar1_model1 = sim1[:,i]
    y_arma_model2 = sim2[:,i]
    betas1.append(calc_ols(y_ar1_model1[1:],y_ar1_model1[0:-1])[0][1])
    betas2.append(calc_ols(y_arma_model2[1:],y_arma_model2[0:-1])[0][1])
print("Mean for AR(1):", np.mean(betas1))
print("Mean for ARMA(2,1):", np.mean(betas2))

## Partial ACF vs. ACF

ACF:
$$
x_{t} =  \boldsymbol{\rho_{j}}\, x_{t-j} + e_{t}
$$

PACF: 
$$
x_{t} =  \phi_{1,j}\, x_{t-1} + ...+ \rho_{j-1,j}\, x_{t-j+1}+ \boldsymbol{\rho_{j,j}}\, x_{t-j} + e_{t}
$$

**Example:** AR(1) $x_{t} =  \phi\, x_{t-1} + e_{t}$

ACF: $\rho_j = \phi^j$

PACF: $\rho_{1,1} = \phi, \quad \rho_{2,2} = \,?$


In [ ]:
### Plotting the Partial Autocorrelation Function 

phi = [0.8]
theta = []

arma_process, phi, theta, lam_phi, lam_theta = arma_from_coeff(phi, theta)

H = 20
_=plt.plot(arma_process.acf(H), marker='o', ms=4)
plt.show()

_=plt.plot(arma_process.pacf(H), marker='o', ms=4)
plt.show()



### Additional exercises
1. Compare ACF and PACF for AR(p) for different p
2. Compute ACF and PACF for MA(q) for different q
3. In practice, how could the shape of the ACF and PACF help you determine the lag orders?

## Simulating ARMA processes

Options:
1. Write your own routine, such as `simulate_arma()` above
2. Use `statsmodels.tsa.arima_process.arma_generate_sample`
3. Use the `generate_sample(nsample=(T,N))` method of the `ArmaProcess` class

In [ ]:
## simulate the same ARMA(1,2) model 10 times
phi = np.array([0.75])
theta = np.array([1.2, -0.4])
arma11, phi, theta, lam_phi, lam_theta = arma_from_coeff(phi, theta)

T, N = 100, 10

sim = arma11.generate_sample(nsample=(T,N))
sim.shape

_=plot_acf(sim[:,0])
_=plot_acf(sim[:,1])

In [ ]:
## Generate simulations for 7 different ARMA models
T = 5000  # simulation length
N = 1     # number of samples
np.random.seed(1234)

## these are the inverse AR and MA roots
lam_phi_list = [
    [0.95], 
    [0.99], 
    [0.9999],
    [0.5], 
    [0.5, 0.2],
    [0.5, 0.45],
    [0.9]
    ]
lam_theta_list = [
    [0], 
    [0],
    [0],
    [0],
    [0],
    [0.55],
    [0]
    ]

ARMA_dict = {}
phi_theta_df = pd.DataFrame()
Samples_dict = {}
Samples_df = pd.DataFrame()
ARMA_acf_df = pd.DataFrame()
for lam_phi, lam_theta, i in zip(lam_phi_list, lam_theta_list, range(1,len(lam_phi_list)+1)):
    print('Case ', i)
    print(lam_phi, lam_theta)
    
    l = arma_from_lambda(lam_phi, lam_theta)  # l is tuple with all return values
    phi_theta_df[i] = [lam_phi, lam_theta, l[1].round(4), l[3].round(4)]
    arma_pr = l[0]  # ArmaProcess
    ARMA_dict[i] = arma_pr  # save model for later
    
    y = pd.DataFrame(arma_pr.generate_sample(nsample=(T,N)))
    Samples_df[i] = y

### adding a seasonality to AR(1) 
#Samples_df[7].iloc[::4] += 2
Samples_df.iloc[::4, 6] += 2
    
#Samples_df.to_csv(path_data+'Lab2_ARMA_simulations.csv')
Samples_df.to_pickle(path_data+'Lab2_ARMA_simulations.pickle')

phi_theta_df.index = ['lam_phi', 'lam_theta', 'phi', 'theta']
#phi_theta_df.to_csv(path_data+'Lab2_ARMA_simulations_true_phi_theta.csv')
phi_theta_df


In [ ]:
### theoretical ACF for each of the seven models

ARMA_acf_df = pd.DataFrame()
for i in ARMA_dict.keys():
    arma_pr = ARMA_dict[i]
    _acf = pd.DataFrame(arma_pr.acf(100))
       
    ARMA_acf_df = pd.concat([ARMA_acf_df, _acf], axis=1)
    
ARMA_acf_df.columns = ARMA_dict.keys()
ARMA_acf_df.to_csv(path_data+'Lab2_ARMA_simulations_true_acf.csv')
ARMA_acf_df.head()

for i in ARMA_acf_df.columns:
    _=plt.plot(ARMA_acf_df.loc[:30,i])
_=plt.legend(ARMA_dict.keys())
_=plt.show()

Load a `pickle` file with the simulation results

In [ ]:
Samples_df = pd.read_pickle(path_data+'Lab2_ARMA_simulations.pickle')
Samples_df.shape
Samples_df.head()

T = Samples_df.shape[0]

In [ ]:
### Plot the simulations
fig, ax = plt.subplots(nrows=7, ncols=1, figsize=(12,16))

for j in Samples_df.columns:
    _=ax[j-1].plot(Samples_df.loc[:,j])
    _=ax[j-1].set_xlabel('Time', fontsize=16)
    _=ax[j-1].set_title('ARMA simulation, case ' + str(j), fontsize=18)
    _=ax[j-1].axhline(y=0, linewidth=0.4)

fig.tight_layout()


In [ ]:
# We will look at ACFs for different sample sizes

T1 = [100, 200, 500, 1000, T]
H = 100

np.random.seed(12345)

acf_df = pd.DataFrame()
pacf_df = pd.DataFrame()
for i in Samples_df.columns:
    _acf_df = pd.DataFrame()
    _pacf_df = pd.DataFrame()
    for t in T1:
        y = Samples_df[i].loc[:t]
        _acf_df[t] = acf(y, nlags=H)
        _pacf_df[t] = pacf(y, nlags=10)
    acf_df = pd.concat([acf_df, _acf_df], axis=1)
    pacf_df = pd.concat([pacf_df, _pacf_df], axis=1)
    
acf_df.columns = pd.MultiIndex.from_product([Samples_df.columns, T1])
pacf_df.columns = pd.MultiIndex.from_product([Samples_df.columns, T1])

acf_df.head()
pacf_df.head()

### Let's study cases 1-3

In [ ]:
### Let's first look at the PACFs of the first three models
# look only at PACF for sample size T=5000
# What do we conclude?

_=pacf_df.loc[:,idx[[1,2,3],T]].droplevel(1,axis=1).plot(marker='o', ms=3.5,title='PACF')

In [ ]:
### Now let's study the ACFs
# Case 1
# ACFs for different sample sizes
i = 1
cases_i = acf_df.loc[:,idx[[i]]]

H = 75
ylim = (-0.5, 1.1)

_=plt.figure()
for t in T1:
    _=plt.plot(cases_i.loc[:,idx[:,t]].droplevel(1, axis=1).loc[:H])
    _=plt.title('Case '+str(i))
    _=plt.legend(T1)
    _=plt.ylim(ylim)
plt.show()

In [ ]:
### Case 2
i = 2
cases_i = acf_df.loc[:,idx[[i]]]

_=plt.figure()
for t in T1:
    _=plt.plot(cases_i.loc[:,idx[:,t]].droplevel(1, axis=1).loc[:H])
    _=plt.title('Case '+str(i))
    _=plt.legend(T1)
    _=plt.ylim(ylim)
plt.show()

In [ ]:
### Case 3
i = 3
cases_i = acf_df.loc[:,idx[[i]]]

_=plt.figure()
for t in T1:
    _=plt.plot(cases_i.loc[:,idx[:,t]].droplevel(1, axis=1).loc[:H])
    _=plt.title('Case '+str(i))
    _=plt.legend(T1)
    _=plt.ylim(ylim)
plt.show()

**Lesson**: Hard to draw firm conclusion from ACFs for small(ish) sample sizes

Now let's compare the 3 cases for a given sample size
1. What do you learn?
2. What sample size is necessary to differentiate the 3 processes?

In [ ]:
### Fix T and plot the ACFs of cases 1-3 in the same figure
### What do you learn?
### 

cases = acf_df.loc[:,idx[[1,2,3]]]
H = 75

for t in T1:
    fig, ax = plt.subplots()
    _p = cases.loc[:,idx[:,t]].droplevel(1, axis=1).loc[:H]
    _=ax.plot(_p)
    _=plt.title('T='+str(t)+', s.e.=$1/\sqrt{T}$= '+str(round(1/t**.5,2)))
    _=plt.legend(_p.columns)
    _=plt.ylim(ylim)
    plt.show()

How do the estimated ACFs with the theoretically correct ACFs?

In [ ]:
ARMA_acf_df.head()
H = 100
for i in ARMA_acf_df.columns[:3]:
    _=plt.plot(ARMA_acf_df.loc[:H,i])
_=plt.legend(ARMA_dict.keys())
_=plt.show()

In [ ]:
H = 75
for i in [1,2,3]:
    _=plt.plot(ARMA_dict[i].acf(H), lw=3)
    _=plt.plot(acf_df.loc[:H,idx[i]])
    _=plt.title('Case '+str(i))
    _=plt.legend(['true']+list(T1))
    plt.show()

In [ ]:
phi_theta_df[[1,2,3]]

### How about cases 4-6?

In [ ]:
H = 20
for i in [4,5,6]:
    # _=plt.plot(ARMA_dict[i].acf(H), lw=4)
    _=plt.plot(acf_df.loc[:H,idx[i,[100,200,500,T]]], lw=2)
    _=plt.title('Case '+str(i))
    _=plt.legend(T1)
    plt.show()

In [ ]:
### Fix T and plot the ACFs in the same figure
### What do we learn?

cases = acf_df.loc[:,idx[[4,5,6]]]
# H = 20

for t in T1:
    fig, ax = plt.subplots()
    _p = cases.loc[:,idx[:,t]].droplevel(1, axis=1).loc[:H]
    _=ax.plot(_p)
    _=plt.title('T='+str(t)+', s.e.=$1/\sqrt{T}$= '+str(round(1/t**.5,2)))
    _=plt.legend(_p.columns)
    _=plt.ylim(ylim)
    plt.show()

In [ ]:
H = 15
for i in [4,5,6]:
    _=plt.plot(ARMA_dict[i].acf(H), lw=3)
    _=plt.plot(acf_df.loc[:H,idx[i,[100,200,500,T]]])
    _=plt.title('Case '+str(i))
    _=plt.legend(['true']+list(T1))
    plt.show()

In [ ]:
phi_theta_df[[4,5,6]]

### Finally, case 7 - seasonality

In [ ]:
_=plt.plot(Samples_df.loc[1:100,7], marker='o', ms=3)

In [ ]:
acf7 = acf_df.loc[:,idx[[7]]].droplevel(0, axis=1)

H = 50
_=plt.plot(acf7[:H], marker='o', ms=3)
_=plt.legend(T1)

In [ ]:
Samples_df[7][0:20]

**Detect seasonality using regressions with seasonal dummies**

In [ ]:
y = Samples_df[7].iloc[:5000]
y.shape

In [ ]:
dummy_dict = {}
for i in range(1,4):
    dummies = np.zeros_like(y)
    dummies[i::12] = 1
    dummy_dict[i] = dummies
dummies12 = pd.DataFrame(dummy_dict)
dummies12.head()
dummies12.shape

In [ ]:
lhs = y.iloc[1:]
rhs = y.iloc[:-1]

res = sm.regression.linear_model.OLS(lhs.values, sm.tools.tools.add_constant(rhs.values)).fit()
print(res.summary())

rhs = pd.concat([rhs,dummies12.iloc[:-1]], axis=1)
res = sm.regression.linear_model.OLS(lhs.values, sm.tools.tools.add_constant(rhs.values)).fit()
print(res.summary())

### Exercise 3

Why is the estimated AR coefficient so far below 0.9? How can we get closer to the "truth"?

## Application: Stock Dividends

Let's analyze **monthly dividends** (in USD ($) nominal billions) from the CRSP-value weighted portfolio return index.

In [ ]:
df_crsp_div = pd.read_csv(path_data+'crspvw_dividends_1927-2022.csv', index_col='date', parse_dates=['date'])
df_crsp_div = df_crsp_div.resample('ME').last()

df_crsp_div.head()
df_crsp_div.tail()

print(df_crsp_div.reset_index().info())

Let's plot the time series of the monthly dividend cash flows from 1980 to the present:

In [ ]:
fig, axes = plt.subplots(figsize=(12,6))
_=axes.plot(df_crsp_div['D'].loc['1980':])
_=axes.set_xlabel('Months', fontsize=18)
_=axes.set_ylabel('nominal dividends', fontsize=18)
_=axes.set_title('Monthly Dividends: CRSP Value-Weighted Portfolio', fontsize=18)
_=axes.axhline(y=0, linewidth=0.4)
fig.tight_layout()

We can see **(1)** **EXPONENTIAL growth** and **(2)** **seasonalities**

Let's address (1) by taking natural log:

In [ ]:
df_crsp_div['log_crsp_div'] = df_crsp_div['D'].apply(np.log)

fig, ax = plt.subplots(figsize=(12,6))
_=ax.plot(df_crsp_div.loc['1970':,'log_crsp_div'], lw=.5)
_=ax.set_xlabel('Months', fontsize=18)
_=ax.set_ylabel('log(nominal Dividends)', fontsize=18)
_=ax.set_title('Monthly Dividends: CRSP Value-Weighted Portfolio', fontsize=18)
# _=axe.axhline(y=0, linewidth=0.4)
fig.tight_layout()

In [ ]:
print(df_crsp_div.loc['2010':,'log_crsp_div'].head(24))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

y = df_crsp_div['log_crsp_div']
decomposition = seasonal_decompose(y)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

seasonal['2020':]

T1 = '2015'

fig = plt.figure(figsize=(10,14))
plt.subplot(411)
plt.plot(y[T1:], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend[T1:], label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal[T1:],label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual[T1:], label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

fig = plt.figure(figsize=(10,14))
plt.subplot(411)
plt.plot(y[T1:], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend[T1:], label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal[T1:],label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual[T1:], label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

Let's compute the *ACF* and *PACF* for the log of CRSP dividends, $log(dividends_{1m})$, and the first difference of the log of CRSP dividends, $\Delta log(dividends_{1m})$:

In [ ]:
df_crsp_div['Dlog_crsp_div'] = df_crsp_div['log_crsp_div'] -\
                               df_crsp_div['log_crsp_div'].shift(1)
df_crsp_div = df_crsp_div.dropna() # drop missings

fig19, axes19 = plt.subplots(figsize=(12,6))
_=axes19.plot(df_crsp_div.loc[',1990':,'Dlog_crsp_div'])
_=axes19.set_xlabel('Months', fontsize=18)
_=axes19.set_ylabel('$\Delta$ log(nominal dividends)', fontsize=18)
_=axes19.set_title('Monthly dividend growth', fontsize=18)
_=axes19.axhline(y=0, linewidth=0.4)
fig19.tight_layout()


In [ ]:
### ACF/PCF for log dividends
nlags=15
fig20, axes20 = plt.subplots(nrows=1, ncols=2, figsize=(16,5))
my_plot_acf(df_crsp_div['log_crsp_div'], nlags, 
            ax=axes20[0], title_string="$log(dividends_{1m,crsp})$", 
            title_fontsize=18, xlabel_string="Months")
my_plot_pacf(df_crsp_div['log_crsp_div'], nlags, 
            ax=axes20[1], title_string="$log(dividends_{1m,crsp})$", 
            title_fontsize=18, xlabel_string="Months")
fig20.tight_layout()

In [ ]:
### ACF/PCF for dividend growth
nlags=15
fig21, axes21 = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
my_plot_acf(df_crsp_div['Dlog_crsp_div'], nlags, 
            ax=axes21[0], title_string="$\Delta log(dividends_{1m,crsp})$", 
            title_fontsize=18, xlabel_string="Months")
my_plot_pacf(df_crsp_div['Dlog_crsp_div'], nlags, 
            ax=axes21[1], title_string="$\Delta log(dividends_{1m,crsp})$", 
            title_fontsize=18, xlabel_string="Months")

fig21.tight_layout()

In [ ]:
### Regression of div growth on lag and dummies

y = df_crsp_div['Dlog_crsp_div']

lhs = y.iloc[1:]
rhs = y.iloc[:-1]

res = sm.regression.linear_model.OLS(lhs.values, sm.api.add_constant(rhs.values)).fit()
print(res.summary())

m = 1
dummy_month = 1*(rhs.index.month==m)[:,None]
for m in range(2,3):
    dm = 1*(rhs.index.month==m)[:,None]
    dummy_month = np.concatenate((dummy_month,dm), axis=1)
dummy_month = pd.DataFrame(dummy_month)
dummy_month.index = rhs.index

rhs_dummies = pd.concat([rhs,dummy_month], axis=1)

res = sm.regression.linear_model.OLS(lhs.values, sm.api.add_constant(rhs_dummies.values)).fit()
print(res.summary())

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# y = df_crsp_div['log_crsp_div']
decomposition = seasonal_decompose(y)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

trend.std(), seasonal.std(), residual.std()
trend.std()/y.std(), seasonal.std()/y.std(), residual.std()/y.std()

seasonal['2020':]

T1 = '2015'

fig = plt.figure(figsize=(10,14))
plt.subplot(411)
plt.plot(y[T1:], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend[T1:], label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal[T1:],label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual[T1:], label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
seasonal_decompose?

Let's remove the seasonality in dividends by aggregating them over the last 12 months, using a rolling sum

$$
D^{12}_t = \sum_{i=0}^{11} D_{t-i}
$$

In [ ]:
df_crsp_div['D12'] = df_crsp_div['D'].rolling(12).sum()
df_crsp_div = df_crsp_div.dropna() # drop missings
#print(df_crsp_div.head())
#print(df_crsp_div.tail())

fig22, axes22 = plt.subplots(figsize=(12,6))
_=axes22.plot(df_crsp_div['D12'])
_=axes22.set_xlabel('Months', fontsize=18)
_=axes22.set_ylabel('nominal dividends', fontsize=18)
_=axes22.set_title('Trailing 12-Month Dividends: CRSP Value-Weighted Portfolio', fontsize=18)
_=axes22.axhline(y=0, linewidth=0.4)
fig22.tight_layout()

Now, we only see **EXPONENTIAL growth**. Let's take logs and first differences of the logs:

In [ ]:
df_crsp_div['log_D12'] = df_crsp_div['D12'].apply(np.log)
df_crsp_div = df_crsp_div.dropna() # drop missings
#print(df_crsp_div.head())
#print(df_crsp_div.tail())

fig23, axes23 = plt.subplots(figsize=(12,6))
_=_=axes23.plot(df_crsp_div['log_D12'])
_=_=axes23.set_xlabel('Months', fontsize=18)
_=_=axes23.set_ylabel('log(nominal dividends)', fontsize=18)
_=_=axes23.set_title('Trailing 12-Month Log Dividends: CRSP Value-Weighted Portfolio', fontsize=18)
# _=axes23.axhline(y=0, linewidth=0.4)
fig23.tight_layout()

We get **LINEAR growth**. The seasonality is gone.

In [ ]:
#df_crsp_div['Dlog_D12'] = df_crsp_div['log_D12'] - df_crsp_div['log_D12'].shift(1) # monthly growth in cumulative dividends
df_crsp_div['Dlog_D12'] = df_crsp_div['log_crsp_div'] - df_crsp_div['log_crsp_div'].shift(12) # year-over-year growth
df_crsp_div = df_crsp_div.dropna() # drop missings
#print(df_crsp_div.head())
#print(df_crsp_div.tail())

fig24, axes24 = plt.subplots(figsize=(12,6))
_=_=axes24.plot(df_crsp_div['Dlog_D12'])
_=_=axes24.set_xlabel('Months', fontsize=18)
_=_=axes24.set_ylabel('$\Delta$ log(nominal dividends)', fontsize=18)
_=_=axes24.set_title('Trailing 12-Month Dividend Growth: CRSP Value-Weighted Portfolio', fontsize=18)
_=_=axes24.axhline(y=0, linewidth=0.4)
fig24.tight_layout()

**Success!** It seems like we have a stationary process, with neither trend nor seasonality.

Now let's compute the *ACF* and *PACF* for the log of our 12-month CRSP dividends, $log(dividends_{12m})$, and the new dividend growth series.

In [ ]:
### ACF/PCF for log 12m-trailing dividends
nlags=15
fig25, axes25 = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
my_plot_acf(df_crsp_div['log_D12'], nlags, 
            ax=axes25[0], title_string="$log(dividends_{12m})$", 
            title_fontsize=18, xlabel_string="Months")
my_plot_pacf(df_crsp_div['log_D12'], nlags, 
            ax=axes25[1], title_string="$log(dividends_{12m})$", 
             title_fontsize=18, xlabel_string="Months")
fig25.tight_layout()

In [ ]:
### ACF/PCF for 12-month dividend growth
nlags=15
fig26, axes26 = plt.subplots(nrows=1, ncols=2, figsize=(16,6))
my_plot_acf(df_crsp_div['Dlog_D12'], nlags, 
            ax=axes26[0], title_string="$\Delta log(dividends_{12m})$", 
            title_fontsize=18, xlabel_string="Months")
my_plot_pacf(df_crsp_div['Dlog_D12'], nlags, 
            ax=axes26[1], title_string="$\Delta  log(dividends_{12m})$", 
             title_fontsize=18, xlabel_string="Months")
fig26.tight_layout()

**Question:** What ARMA($p$,$q$) processes do both $log(dividends_{12m})$ and $\Delta log(dividends_{12m})$ follow?

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

y = df_crsp_div['Dlog_D12']
decomposition = seasonal_decompose(y)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

trend.std(), seasonal.std(), residual.std()
trend.std()/y.std(), seasonal.std()/y.std(), residual.std()/y.std()

seasonal['2020':]

T1 = '2015'

fig = plt.figure(figsize=(10,14))
plt.subplot(411)
plt.plot(y[T1:], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend[T1:], label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal[T1:],label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual[T1:], label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

### Exercise 4: Changing the frequency

An alternative to 12-month trailing sums is to use *annual* observations of dividends, e.g., dividends in December of every year.  Repeat the analysis with annual dividend observations and compare the results to those we got with MA(12)-dividends.

### Conclusions: What have we learned?

1. Dividends are highly seasonal
2. Removing seasonality is difficult
3. Some ways to solve the problem.

## (Optional) Parsimonious MA($1$) model

Let's simulate the following MA($1$) model:
$$
y_t = \varepsilon_t - \varepsilon_{t-1} 
$$

What is a particuarly parsimonious way in which we can transform this model?

In [ ]:
### Simulate
np.random.seed(5)
ysim = simulate_arma(ma=[-1], nsample=500).squeeze()

### Note that differencing doesn't help
fig27, axes27 = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
for d in range(2):
    if d==0:
        Dysim = np.copy(ysim)
        axes27[d,0].plot(Dysim)
    else:
        Dysim = Dysim[1:] - Dysim[:-1]
        axes27[d,0].plot(Dysim[:200])
    axes27[d,0].set_title('Diff = %d'% d, fontsize=18)
    ### ACF/PACF
    my_plot_acf(y=Dysim, nlags=10 , ax=axes27[d,1], title_fontsize=18)
    my_plot_pacf(y=Dysim, nlags=10 , ax=axes27[d,2], title_fontsize=18)
    
fig27.tight_layout()

By cumulating the $y_t$ process over time, we can get a much more parsimonious representation! 

In [ ]:
### Let's instead consider the cumulative sum: {sum from 0 to t}y_t = e_t
y_cumsum = np.cumsum(ysim) #https://docs.scipy.org/doc/numpy/reference/generated/numpy.cumsum.html

fig28, axes28 = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
axes28[0].plot(y_cumsum)
axes28[0].set_xlabel('Time', fontsize=16)
axes28[0].set_title("Cumulative Sum of $\{y_t\}_{t\geq0}$", fontsize=18)

### ACF/PACF
my_plot_acf(y=y_cumsum, nlags=10 , ax=axes28[1], title_fontsize=18)
my_plot_pacf(y=y_cumsum, nlags=10 , ax=axes28[2], title_fontsize=18)

fig28.tight_layout()